In [1]:
import requests
import json
import pandas as pd
from fake_useragent import UserAgent

In [2]:
def merge_dfs(df1, df2, key, type_of_inp):
    merged_df = pd.merge(df1, df2, on=key, how=type_of_inp)
    return merged_df

In [3]:
temp_user_agent = UserAgent()
browser_header = {'User-Agent': temp_user_agent.random}

In [4]:
state_id_mapping_response = requests.get("https://cdn-api.co-vin.in/api/v2/admin/location/states",
                                         headers=browser_header)

In [5]:
state_to_id_mapping_df = pd.DataFrame(json.loads(state_id_mapping_response.text)["states"])

In [7]:
state_to_id_mapping_df.head()

,state_id,state_name
0,1,Andaman and Nicobar Islands
1,2,Andhra Pradesh
2,3,Arunachal Pradesh
3,4,Assam
4,5,Bihar


In [8]:
max_state_number = state_to_id_mapping_df["state_id"].max()

In [9]:
state_mapping_list = []
for state_code in range(1,max_state_number+1):
    print("State code: ", state_code)
    response = requests.get(f"https://cdn-api.co-vin.in/api/v2/admin/location/districts/{state_code}",  
                            headers=browser_header)
    json_data = json.loads(response.text)
    for i in json_data["districts"]:
        state_mapping_list.append([state_code, i["district_id"], i["district_name"]])

State code:  1
State code:  2
State code:  3
State code:  4
State code:  5
State code:  6
State code:  7
State code:  8
State code:  9
State code:  10
State code:  11
State code:  12
State code:  13
State code:  14
State code:  15
State code:  16
State code:  17
State code:  18
State code:  19
State code:  20
State code:  21
State code:  22
State code:  23
State code:  24
State code:  25
State code:  26
State code:  27
State code:  28
State code:  29
State code:  30
State code:  31
State code:  32
State code:  33
State code:  34
State code:  35
State code:  36
State code:  37


In [10]:
df = pd.DataFrame(state_mapping_list, columns=["state_id",
                                               "district id", 
                                               "district name"])

In [12]:
df.head()

,state_id,district id,district name
0,1,3,Nicobar
1,1,1,North and Middle Andaman
2,1,2,South Andaman
3,2,9,Anantapur
4,2,10,Chittoor


In [13]:
final_df = merge_dfs(df, 
                     state_to_id_mapping_df, 
                     "state_id", 
                     "left")

In [19]:
final_df.head()

,state_id,district id,district name,state_name
0,1,3,Nicobar,Andaman and Nicobar Islands
1,1,1,North and Middle Andaman,Andaman and Nicobar Islands
2,1,2,South Andaman,Andaman and Nicobar Islands
3,2,9,Anantapur,Andhra Pradesh
4,2,10,Chittoor,Andhra Pradesh


In [18]:
final_df.rename({"state_id" : "state id",
                 "state_name" : "state name"
                },
               inplace=True)

In [20]:
final_df.to_csv("district_mapping.csv", index=False)